In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [37]:
#loading the dataset
df=pd.read_excel(r"C:\Users\abdulmuiz.adewale\OneDrive - Tolaram Pte Ltd\Documents\Indomie Projection.xlsx")
df.head()

,Brands,Month,Year,Target,Achievements
0,Indomie,Jan,2023,475000,406728
1,Indomie,Feb,2023,500000,365165
2,Indomie,Mar,2023,510000,455269
3,Indomie,Apr,2023,525000,455066
4,Indomie,May,2023,535000,450809


In [38]:
#converting month name to number
month_map= {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 
            'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11 ,'Dec':12
            }
df['Month']=df['Month'].map(month_map)

In [39]:
df.head()

,Brands,Month,Year,Target,Achievements
0,Indomie,1,2023,475000,406728
1,Indomie,2,2023,500000,365165
2,Indomie,3,2023,510000,455269
3,Indomie,4,2023,525000,455066
4,Indomie,5,2023,535000,450809


In [40]:
#Feature Engineering
df["Achievement Ratio"]= df['Achievements']/df['Target']

In [42]:
#Brand encoding
#df=pd.get_dummies(df, columns=['Brand'])

#feature and targets
features=['Month', 'Year', 'Achievements', 'Achievement Ratio'] 

X= df[features]
y =df['Target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluate
preds = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, preds))

# Save your trained model (optional)
import joblib
joblib.dump(model, 'target_predictor_model.pkl')

MAE: 29870.0


['target_predictor_model.pkl']

# Updating the prediction function

In [73]:
def predict_next_target(month, year, df, model):
    #converting month name to number
    month_map= {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 
            'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11 ,'Dec':12
            }
    month_number = month_map[month]
    df['Month'] = df['Month'].map(month_map)


    # Get previous year's achievement for the same month
    prev_year = year - 1
    match = df[(df['Year'] == prev_year) & (df['Month'] == month_number)]

    if match.empty:
        raise ValueError(f"No data found for {month} {prev_year} to use as past achievement.")

    achievement = match['Achievements'].values[0]
    target_last_year = match['Target'].values[0]
    achievement_ratio = achievement / target_last_year

    #preparing input
    input_data= pd.DataFrame([{

        'Month':month_number,
        'Year': year,
        'Achievements': achievement,
        'Achievement Ratio': achievement_ratio
    }])

    #Prediction
    predict_target= model.predict(input_data)[0]
    return round(predict_target)

## USAGE

In [74]:
import joblib

In [95]:
#Loading the model
df = pd.read_excel(r"C:\Users\abdulmuiz.adewale\OneDrive - Tolaram Pte Ltd\Documents\Indomie Projection.xlsx")
model = joblib.load('target_predictor_model.pkl')

#predict
month='Dec'
year=2024
predicted= predict_next_target(month=month, year=year, df=df, model=model)
print(f"Predicted Target for {month} {year}: {predicted}")

Predicted Target for Dec 2024: 574550


In [90]:
df.tail(13)

,Brands,Month,Year,Target,Achievements
11,Indomie,12,2023,475000,284803
12,Indomie,1,2024,475000,453143
13,Indomie,2,2024,500000,403279
14,Indomie,3,2024,510000,113931
15,Indomie,4,2024,525000,241301
16,Indomie,5,2024,535000,393219
17,Indomie,6,2024,550000,405219
18,Indomie,7,2024,565000,453417
19,Indomie,8,2024,575000,437282
20,Indomie,9,2024,590000,456113
